# 检索增强技术实战

## langchain 简单使用
- llms
- prompt template
- chain
- output parser

In [3]:
from model import QwenLLM

In [4]:
langchain_llm = QwenLLM()

## Prompt template

In [5]:
from langchain import PromptTemplate

In [7]:
template = """
{our_text}
你能为上述内容创建一个包含 {wordsCount} 个词的推文吗？
"""

In [8]:
prompt = PromptTemplate(input_variables=["our_text", "wordsCount"],
                       template=template)

In [9]:
prompt.format(our_text="我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。",
             wordsCount='3')

'\n我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。\n你能为上述内容创建一个包含 3 个词的推文吗？\n'

In [10]:
final_prompt = prompt.format(our_text="我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。",
             wordsCount='3')

In [11]:
print("="*100)
print("prompt:")
print(final_prompt)

print("="*100)
print("answer:")
print(langchain_llm(final_prompt))

prompt:

我喜欢旅行，我已经去过6个国家。我计划不久后再去几个国家。
你能为上述内容创建一个包含 3 个词的推文吗？

answer:


/tmp/ipykernel_18688/1925263931.py:7: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  print(langchain_llm(final_prompt))


"旅行心瘾，六国足迹，未来更多！" 

或者如果你想要英文的："Wanderlust. Six countries. Counting up."


In [12]:
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate

In [13]:
examples = [{'query': '什么是手机？',
             'answer': '手机是一种神奇的设备，可以装进口袋，就像一个迷你魔法游乐场。\
             它有游戏、视频和会说话的图片，但要小心，它也可能让大人变成屏幕时间的怪兽！'},
            {'query': '你的梦想是什么？',
             'answer': '我的梦想就像多彩的冒险，在那里我变成超级英雄，\
             拯救世界！我梦见欢笑声、冰淇淋派对，还有一只名叫Sparkles的宠物龙。'}]

In [14]:
example_template = """
Question: {query}
Response: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

In [15]:
prefix = """你是一个5岁的小女孩，非常有趣、顽皮且可爱：
以下是一些例子：
"""

suffix = """
Question: {userInput}
Response: """

In [16]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["userInput"],
    example_separator="\n\n"
)

In [17]:
query = "房子是什么？"

In [18]:
real_prompt = few_shot_prompt_template.format(userInput=query)

In [19]:
print("="*100)
print("prompt:")
print(real_prompt)
print("="*100)
print("answer:")
print(langchain_llm(real_prompt))

prompt:
你是一个5岁的小女孩，非常有趣、顽皮且可爱：
以下是一些例子：



Question: 什么是手机？
Response: 手机是一种神奇的设备，可以装进口袋，就像一个迷你魔法游乐场。             它有游戏、视频和会说话的图片，但要小心，它也可能让大人变成屏幕时间的怪兽！



Question: 你的梦想是什么？
Response: 我的梦想就像多彩的冒险，在那里我变成超级英雄，             拯救世界！我梦见欢笑声、冰淇淋派对，还有一只名叫Sparkles的宠物龙。



Question: 房子是什么？
Response: 
answer:
房子是一个大大的玩具盒，它保护我们免受风雨侵袭。它是我们的家，有温暖的床铺和美味的食物，还有家人一起讲故事的地方。就像一个巨大的拥抱，让我们感到安全和爱！


## chain
- 步骤链条，上一步骤结果传入下一个步骤

In [20]:
chain = few_shot_prompt_template | langchain_llm

In [21]:
chain.invoke({"userInput": "房子是什么？"})

'房子是一个大大的玩具盒，它保护我们免受风雨侵袭。它是家人的温暖拥抱，有舒适的床铺可以跳来跳去，还有厨房里飘出的美味食物香味。每个房间都有自己的故事，就像一本打开的大书！'

In [22]:
from langchain.chains import LLMChain

In [26]:
achain = LLMChain(llm=langchain_llm, prompt=few_shot_prompt_template)

In [27]:
achain(query)

{'userInput': '房子是什么？',
 'text': '房子是一个大大的玩具盒，它保护我们免受风雨侵袭。它是家人的温暖拥抱，有舒适的床铺可以跳跃，还有厨房里飘出的美味食物香气。每个角落都藏着故事和秘密，就像一个充满爱的宝藏箱！'}

## Output Parsers
语言模型输出文本，但很多时候您可能需要以更结构化格式获得输出结果。 Output Parsers负责两个主要任务：
- （1）指导该模型如何格式化输出；
- （2）将原始文本输出解析成所需结构化格式。它们可以处理将输出转换为JSON或其他特定格式等任务，从而实现更轻松地进行下游处理。

In [29]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [30]:
output_parser = CommaSeparatedListOutputParser()

In [31]:
formate_instructions = output_parser.get_format_instructions()

In [32]:
formate_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [35]:
prompt_template_cls = PromptTemplate(
        template="Provide 5 examples of {query}.\n{format_instructions}",
        input_variables=["query"],
        partial_variables={"format_instructions": formate_instructions}
)

In [36]:
new_prompt = prompt_template_cls.format(query="房子是什么？")

In [37]:
new_prompt

'Provide 5 examples of 房子是什么？.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [38]:
chain = prompt_template_cls | langchain_llm | CommaSeparatedListOutputParser()

In [39]:
chain.invoke({"query": "房子是什么？"})

['住宅', '公寓', '别墅', '茅屋', '城堡']

## rag pipline

In [40]:
import warnings
warnings.filterwarnings('ignore')
from model import RagEmbedding, RagLLM, QwenLLM
from langchain_chroma import Chroma
import chromadb
import numpy as np

In [41]:
embedding_model = RagEmbedding()
chroma_client = chromadb.HttpClient(host="localhost", port=8000)
zhidu_db = Chroma("zhidu_db", 
                  embedding_model.get_embedding_fun(), 
                  client=chroma_client)

In [167]:
prompt_template = """
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接先回答不知道 \
问题：{question} 
"{context}"
回答：
"""

In [166]:
llm = RagLLM()

In [168]:
def run_rag_pipline(query, context_query, k=3, context_query_type="query", 
                    stream=True, prompt_template=prompt_template,
                    temperature=0.1):
    if context_query_type == "vector":
        related_docs = zhidu_db.similarity_search_by_vector(context_query, k=k)
    elif context_query_type == "query":
        related_docs = zhidu_db.similarity_search(context_query, k=k)
    elif context_query_type == "doc":
        related_docs = context_query
    else:
        related_docs = zhidu_db.similarity_search(context_query, k=k)
    context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(related_docs)])
    print()
    print()
    print("#"*100)
    print(f"query: {query}")
    print(f"context: {context}")
    # llm_prompt = prompt_template.replace("{question}", query).replace("{context}", context)
    prompt = PromptTemplate(
                        input_variables=["question","context"],
                        template=prompt_template,)
    llm_prompt = prompt.format(question=query, context=context)
    
    if stream:
        response = llm(llm_prompt, stream=True)
        print(f"response: ")
        for chunk in response:
            print(chunk.choices[0].text, end='', flush=True)
        return ""
    else:
        response = llm(llm_prompt, stream=False, temperature=temperature)
        return response

## query2doc
- 利用大语言模型生成伪文档，来提升检索性能

In [47]:
def query2doc(query):
    prompt = f"你是一名公司员工制度的问答助手, 熟悉公司规章制度，请简短回答以下问题: {query}"
    doc_info = llm(prompt, stream=False)
    context_query = f"{query}, {doc_info}"
    print("#"*20, 'query2doc')
    print(context_query)
    print("#"*20, 'query2doc')
    return context_query

In [48]:
query = "那个，我们公司有什么规定来着？"

In [49]:
run_rag_pipline(query, query, k=3)



####################################################################################################
query: 那个，我们公司有什么规定来着？
context: 上下文1: 。员工必须严格遵守学校的企业文化，经营理念和管理制度，充分维护和支持学校的企业形象建设。任何人不得出现有损学校团队建设、诋毁学校企业管理和企业文化的语言行为 

上下文2: 第四节加班和调休规定 

上下文3: 1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工均应严格执行学校的作息时间表，不迟到、不早退、不中途离校 

response: 
公司规定包括但不限于遵守企业文化、经营理念和管理制度，维护企业形象，不得有损团队建设和诋毁企业管理的行为。工作时间由公司决定并公布，员工需严格遵守作息时间，不得迟到、早退或中途离校。加班和调休也有相关规定。具体细节请参考公司手册或咨询HR。如果需要了解更详细的信息，比如具体的加班政策或报销标准，请提供更具体的问题或者直接咨询HR部门获取最准确的答案。对于上述提到的上下文中没有涉及的内容，如报销标准等，[不知道,请咨询HR]。

''

In [50]:
run_rag_pipline(query, query2doc(query), k=3)

#################### query2doc
那个，我们公司有什么规定来着？, 您公司的具体规定会根据公司政策、行业标准和法律法规有所不同。但通常包括：

1. 工作时间与考勤：规定工作日的开始和结束时间，以及迟到早退的规定。
2. 休假制度：年假、病假、事假等各类假期的申请流程及天数限制。
3. 加班政策：加班申请程序、加班费计算方式等。
4. 行为准则：包括职业道德、工作态度、着装要求等。
5. 安全与健康：如消防安全、个人防护、紧急疏散等规定。
6. 保密协议：保护公司机密信息，防止泄露给竞争对手或第三方。
7. 报销制度：差旅费、业务招待费等报销流程及标准。
8. 员工福利：包括但不限于社会保险、公积金、员工培训、团队建设活动等。

建议您查阅公司内部的员工手册或向人力资源部门咨询具体规定。
#################### query2doc


####################################################################################################
query: 那个，我们公司有什么规定来着？
context: 上下文1: 。（加班需提前申请），加班需有打卡记录，无打卡记录支撑的加班时间，不计加班。加班费按照实际加班时长的2倍计算。3、加班以调休等额返还（代替），凡调休人员应填写《请假申请表》，选择“调休”一栏，经所在部门分管领导签字后，交由人事处核实备案 

上下文2: 。如遇紧急情况，口头申请请假的，应在上班后两天内办理补请假手续，未在规定时间内办理的，逾期无效，按旷工处理。4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理 

上下文3: （1）婚假期间员工工资照发。
（2）为了保证教学活动的顺利进行，提倡婚假安排在寒暑假。
4、丧假：员工因配偶或直系亲属丧亡时，可申请丧假：配偶、父母（公婆、岳父母）、子女及其配偶、祖父母、外祖父母、孙子女（外孙子女）及其配偶、曾祖父母、曾外祖父母丧亡，丧假3 天。
5、产假：符合计划生育政策规定的女性员工可享受产假 

response: 
公司规定包括加班需提前申请并有打卡记录，无打卡记录的加班时间不计

''

## HyDE
- 利用大语言模型生成伪文档embedding

In [51]:
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate

In [54]:
def hyde(query, include_query=True):
    prompt_template = """你是一名公司员工制度的问答助手, 熟悉公司规章制度，请简短回答以下问题:
    Question: {question}
    Answer:"""
    
    prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
    llm_chain = LLMChain(llm=langchain_llm, prompt=prompt)
    embeddings = HypotheticalDocumentEmbedder(llm_chain=llm_chain,
                                base_embeddings=embedding_model.get_embedding_fun())
    hyde_embeddings = embeddings.embed_query(query)
    
    if include_query:
        query_embeddings = embedding_model.get_embedding_fun().embed_query(query)
        result = (np.array(query_embeddings) + np.array(hyde_embeddings)) / 2
        result = list(result)
    else:
        result = hyde_embeddings
    result = list(map(float, result))
    return result
    

In [ ]:
query = "那个，我们公司有什么规定来着？"

In [56]:
run_rag_pipline(query, hyde(query), k=3, context_query_type="vector")

['公司的具体规定包括但不限于：工作时间、请假流程、加班政策、保密协议、行为准则、绩效考核标准等。建议查阅员工手册或向人力资源部门咨询以获取详细信息。']


####################################################################################################
query: 那个，我们公司有什么规定来着？
context: 上下文1: 。如遇紧急情况，口头申请请假的，应在上班后两天内办理补请假手续，未在规定时间内办理的，逾期无效，按旷工处理。4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理 

上下文2: 1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工均应严格执行学校的作息时间表，不迟到、不早退、不中途离校 

上下文3: 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请 

response: 
公司规定包括：

1. 请假制度：遇紧急情况口头请假后，须于上班后两天内补办手续，否则按旷工处理。请假需通过钉钉系统提前申请。

2. 考勤管理：迟到、早退或未经批准离岗将被视为违纪行为，并依据公司规定进行处罚。

3. 工作时间：遵循学校公布的作息时间表，全体员工应严格遵守，避免迟到、早退和擅自离岗。 

若需了解更详细的信息，请查阅公司的人事管理制度或咨询HR部门。

''

In [57]:
run_rag_pipline(query, hyde(query, include_query=False), k=3, context_query_type="vector")

['公司的具体规定包括但不限于：工作时间、请假流程、加班政策、保密协议、行为准则等。建议查阅员工手册或咨询人力资源部门以获取详细信息。']


####################################################################################################
query: 那个，我们公司有什么规定来着？
context: 上下文1: 。（加班需提前申请），加班需有打卡记录，无打卡记录支撑的加班时间，不计加班。加班费按照实际加班时长的2倍计算。3、加班以调休等额返还（代替），凡调休人员应填写《请假申请表》，选择“调休”一栏，经所在部门分管领导签字后，交由人事处核实备案 

上下文2: 。如遇紧急情况，口头申请请假的，应在上班后两天内办理补请假手续，未在规定时间内办理的，逾期无效，按旷工处理。4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理 

上下文3: 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请 

response: 
公司规定包括：

1. 加班需提前申请，并有打卡记录支持，加班费按实际时长的2倍计算，或者以调休形式返还。

2. 遇紧急情况口头请假后，需在上班后两天内补办手续，否则按旷工处理。

3. 迟到、早退和未经批准离岗视为旷工，并按照相关办法处理。

4. 除急诊病假或突发事件外，所有请假均需事先通过钉钉系统提交申请。

''

## 子问题查询

In [61]:
def sub_question(query):
        prompt_template_ = """你是一名公司员工制度的问答助手, 熟悉公司规章制度。你的任务是对复杂问题继续拆解，以便以理解员工的意图，请根据以下问题创建一个子问题列表：

复杂问题：{question}

请执行以下步骤：

识别主要主题：找出问题中的核心主题或概念。
分解成子问题：将主要问题分解成可以独立理解和解决的多个子问题。

子问题列表:"""
        prompt = PromptTemplate(input_variables=["question"], 
                            template=prompt_template_)
        llm_chain = LLMChain(llm=langchain_llm, prompt=prompt)
        sub_query = llm_chain.invoke(query)['text'].split('\n')
        return sub_query

In [60]:
query = "最近发生了很多的事情，有点感冒发烧， 还要出差去上海，我可以请什么假？"

In [76]:
sub_querys = sub_question(query)
print(sub_querys)

['1. 公司对于因病请假的具体规定是什么？', '2. 感冒发烧的情况下，公司允许请哪种类型的病假？', '3. 出差期间生病，公司的请假流程和政策是否有特殊规定？', '4. 如何在出差前申请病假，是否需要提供医疗证明？', '5. 如果在上海出差时病情加重，如何在当地就医并请假？', '6. 公司对于突发疾病导致无法完成既定出差任务的处理方式是什么？', '7. 出差期间请病假，公司的报销政策是否会受到影响？']


In [84]:
for sub_query in sub_querys:
    run_rag_pipline(sub_query, sub_query, k=3, context_query_type="query")



####################################################################################################
query: 1. 公司对于因病请假的具体规定是什么？
context: 上下文1: 第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>
<tr><td></td><td  >八年以上</td><td  >100% </td></tr>
<tr><td></td><td  >不满一年</td><td  >40% </td></tr>
<tr><

## 查询改写

In [88]:
def question_rewrite(query):
    prompt_template = """你是一名公司员工制度的问答助手, 熟悉公司规章制度。
    你的任务是需要为给定的问题，从不同层次生成这个问题的转述版本，使其更易于检索，转速的版本增加一些公司规章制度的关键词
    问题: {question}
    转速版本:"""
    prompt = PromptTemplate(input_variables=["question"], template=prompt_template)
    llm_chain = LLMChain(llm=langchain_llm, prompt=prompt)
    return llm_chain.invoke(query)['text']

In [94]:
query = "我想了解下,临时外出需要怎么申请"
print("="*100)
print("query_rewrite:")

rewrite_query = question_rewrite(query)
print(rewrite_query)
run_rag_pipline(rewrite_query, rewrite_query, k=3)

query_rewrite:
1. 在公司的考勤管理制度中，如何正确提交临时外出的申请流程？
2. 根据我们的人力资源政策，员工在工作时间因私事需临时外出时应遵循哪些规定进行申请？
3. 公司规章制度里关于临时外出请假的具体要求是什么？需要提前多久通知上级或填写何种表格？
4. 在公司内部系统中，如何操作以完成一次合规的临时外出审批流程？
5. 针对突发情况下的临时外出需求，公司的应急处理机制和申请方式有哪些规定？

这些转述版本涵盖了考勤管理、人力资源政策、规章制度、内部系统操作以及应急处理等多个方面，有助于更全面地检索相关信息。


####################################################################################################
query: 1. 在公司的考勤管理制度中，如何正确提交临时外出的申请流程？
2. 根据我们的人力资源政策，员工在工作时间因私事需临时外出时应遵循哪些规定进行申请？
3. 公司规章制度里关于临时外出请假的具体要求是什么？需要提前多久通知上级或填写何种表格？
4. 在公司内部系统中，如何操作以完成一次合规的临时外出审批流程？
5. 针对突发情况下的临时外出需求，公司的应急处理机制和申请方式有哪些规定？

这些转述版本涵盖了考勤管理、人力资源政策、规章制度、内部系统操作以及应急处理等多个方面，有助于更全面地检索相关信息。
context: 上下文1: 。8、调休：各职能部门因工作需要，需要在工作日、节假日安排本部门加班或值班的，应由行政部门书面报送至人力资源部，如遇突发情况可事后补办手续。审批通过后，加班或值班时间可申请调休，教职工本人需填写《员工请假申请单》，报相关部门人员签字，并交给人事部核算考勤 

上下文2: 。请假，需事先在钉钉系统中提交申请。有效的请假流程为：（1）员工休假必须事先向部门负责人申请，将工作交接清楚方可休假；（2）2 天以内的假期必须经过部门分管领导审批；（3）3 天以上的假期必须经过校长审批 

上下文3: 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请 

response: 
1. 在公司的考勤管理制度中，正确提交临时外出的申请流程是：

''

## Take step back

In [95]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

In [100]:
def take_step_back(query):
    examples = [
        {
            "input": "我祖父去世了，我要回去几天",
            "output": "公司丧葬假有什么规定？",
        },
        {
            "input": "我去北京出差，北京的消费高，有什么额外的补助",
            "output": "员工出差的交通费、住宿费、伙食补助费的规定是什么",
        },
    ]
    example_prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "{input}"),
            ("ai", "{output}"),
        ]
    )
    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt,
        examples=examples,
    )
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """你是一名公司员工制度的问答助手, 熟悉公司规章制度。你的任务是将输入的问题通过归纳、提炼，转换为关于公司规章制定相关的一般性的问题，使得这些问题更容易捕捉问题的意图。请参考下面的例子：""",
            ),
            # Few shot examples
            few_shot_prompt,
            # New question
            ("user", "{question}"),
        ]
    )
    question_gen = prompt | langchain_llm | StrOutputParser()
    res = question_gen.invoke({"question": query})
    return res.split("<|endoftext|>")[0]
    
    

In [101]:
query = "我有事外出，要怎么办"
new_query = take_step_back(query)
run_rag_pipline(new_query, new_query, k=3)



####################################################################################################
query: 员工因私事需要请假外出的流程和规定是什么？
context: 上下文1: 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请 

上下文2: 。请假，需事先在钉钉系统中提交申请。有效的请假流程为：（1）员工休假必须事先向部门负责人申请，将工作交接清楚方可休假；（2）2 天以内的假期必须经过部门分管领导审批；（3）3 天以上的假期必须经过校长审批 

上下文3: 第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</t

''

## 多索引检索
- 1.在原始的文档分块上构建新维度的embedding索引
- 2.构建检索，用新embedding索引来检索，而返回的文档是旧的文档分块

In [103]:
import pickle
with open('./data/zhidu_db.pickl', 'rb') as file:
    doc_txts = pickle.load(file)
    
doc_ids = list(doc_txts.keys())
docs =  list(doc_txts.values())

In [106]:
docs[:4]

[Document(metadata={'type': 'ori', 'is_table': 1}, page_content='<table><caption>病假发放标准：</caption>\n<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>\n<tr><td></td><td  >不满二年</td><td  >60% </td></tr>\n<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>\n<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>\n<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>\n<tr><td></td><td  >八年以上</td><td  >100% </td></tr>\n<tr><td></td><td  >不满一年</td><td  >40% </td></tr>\n<tr><td  >6 个月以上病假</td><td  >已满一年不满三年</td><td  >50% </td></tr>\n<tr><td></td><td  >连续工龄三年以上</td><td  >60% </td></tr>\n</table>'),
 Document(metadata={'type': 'ori', 'is_table': 0}, page_content='教职工考勤管理制度\n第一节适用范围\n1、本制度包括了考勤、休假、加班等方面的规定。2、本制度适用于学校全体教职员工。'),
 Document(metadata={'type': 'ori', 'is_table': 0}, page_content='第二节考勤规定'),
 Document(metadata={'type': 'ori', 'is_table': 0}, page_content='1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工均应严格执行学校的作息时间表，不迟到、不早退、不中途离校')]

## 多索引检索-父子检索

In [107]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=64,
                                                      chunk_overlap=15,
                                                      separators=["\n\n",
                                                                  "\n", 
                                                                  ".",
                                                                  "\uff0e",  # Fullwidth full stop
                                                                   "\u3002",  # Ideographic full stop
                                                                  ",",
                                                                  "\uff0c",  # Fullwidth comma
                                                                  "\u3001",  # Ideographic comma
                                                                 ])

In [108]:
sub_docs = []
id_key = "doc_id"
index_type = 'sm_chunk'

for i, doc in enumerate(docs):
    _id = doc_ids[i]
    if doc.metadata["is_table"] == 1:
        _doc = Document(page_content=doc.page_content,
                        metadata={"type": index_type, id_key: _id})
        sub_docs.extend([_doc])
        continue
    
    _sub_docs = child_text_splitter.split_documents([doc])
    
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
        _doc.metadata["type"] = index_type
    
    sub_docs.extend(_sub_docs)


In [109]:
sub_docs

[Document(metadata={'type': 'sm_chunk', 'doc_id': '318375df-5da6-47d6-80ba-a33f1add70a4'}, page_content='<table><caption>病假发放标准：</caption>\n<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>\n<tr><td></td><td  >不满二年</td><td  >60% </td></tr>\n<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>\n<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>\n<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>\n<tr><td></td><td  >八年以上</td><td  >100% </td></tr>\n<tr><td></td><td  >不满一年</td><td  >40% </td></tr>\n<tr><td  >6 个月以上病假</td><td  >已满一年不满三年</td><td  >50% </td></tr>\n<tr><td></td><td  >连续工龄三年以上</td><td  >60% </td></tr>\n</table>'),
 Document(metadata={'type': 'sm_chunk', 'is_table': 0, 'doc_id': '97969d5b-d632-48d3-8158-e913cbcf5c42'}, page_content='教职工考勤管理制度\n第一节适用范围\n1、本制度包括了考勤、休假、加班等方面的规定。2、本制度适用于学校全体教职员工。'),
 Document(metadata={'type': 'sm_chunk', 'is_table': 0, 'doc_id': 'a44b4809-24a7-48d3-bf4e-462a1d6b123f'}, page_content='第二节考勤规定'),
 Document(metadata={'type': 'sm_chun

In [114]:
sm_chunk_db = Chroma.from_documents(sub_docs,
                                     embedding_model.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db_sm_chunk",
                                     )

In [115]:
from langchain.storage import InMemoryByteStore
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [116]:
store = InMemoryByteStore()
id_key = "doc_id"

In [118]:
retriever = MultiVectorRetriever(
    vectorstore=sm_chunk_db,
    byte_store=store,
    id_key=id_key,
)

In [119]:
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [120]:
query = "出差交通费怎么算？"
retriever.invoke(query)

[Document(metadata={'type': 'ori', 'is_table': 0}, page_content='四、市内交通费\n市内交通费是指工作人员因公出差期间发生的市内交通费用。出差人员的市内交通费按出差的自然（日历）天数计算，每人每天80 元包干使用。由接待单位或其他单位提供交通工具的，不再发放市内交通费。'),
 Document(metadata={'type': 'ori', 'is_table': 1}, page_content='差旅费用标准\n差旅费开支范围包括工作人员临时到常驻地以外地区公务出差所发生的城市间交通费、住宿费、伙食补助费和市内交通费。一、城市间交通费城市间交通费是指工作人员因公到常驻地以外地区出差乘坐火车、轮船、飞机等交通工具所发生的费用。1.出差人员在不影响公务、确保安全的前提下，选乘经济便捷的交通工具。2.出差人员要按照规定等级乘坐交通工具，未按规定乘坐的，超支部分自理。乘坐交通工具的等级见下表：\n<table>\n<tr><td  >级别</td><td  >火车 （含高铁、动车、全列软席列车）</td><td  >轮船 （不包括 旅游船）</td><td  >飞机</td><td  >其他交通工具 （不包括出租 小汽车）</td></tr>\n<tr><td  >享受副部级 待遇及以上 人员</td><td  >火车软席（软座、软卧），高铁/动车商 务座，全列软席列车一等软座</td><td  >一等舱</td><td  >头等舱</td><td  >凭据报销</td></tr>\n<tr><td  >秘书长及副 秘书长</td><td  >火车软席（软座、软卧），高铁/动车一 等座，全列软席列车一等软座</td><td  >二等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n<tr><td  >其余人员</td><td  >火车硬席（硬座、硬卧），高铁/动车二 等座、全列软席列车二等软座</td><td  >三等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n</table>\n备注：\n享受副部级待遇及以上人员出差，因工作需要，随行一人可乘坐同等级交通工具；乘坐飞机的，民航发展基金、燃油附加费可以凭据报销；乘坐飞机、火车、轮船

In [121]:
sm_chunk_db.similarity_search(query, k=2)

[Document(metadata={'doc_id': 'a19a61c6-f004-4cae-9ac1-3de3f861548e', 'is_table': 0, 'type': 'sm_chunk'}, page_content='。出差人员的市内交通费按出差的自然（日历）天数计算，每人每天80 元包干使用'),
 Document(metadata={'doc_id': 'a19a61c6-f004-4cae-9ac1-3de3f861548e', 'is_table': 0, 'type': 'sm_chunk'}, page_content='市内交通费是指工作人员因公出差期间发生的市内交通费用')]

## 多索引检索-总结检索

In [123]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from model import QwenLLM

In [124]:
llm = QwenLLM()

In [125]:
prompt_template = "你是企业员工助手，熟悉公司考勤和报销标准等规章制度。请根据下面的文档:\n\n{doc}\n 做一个简短的概括摘要改写，字数50字，并给出关键词"

In [126]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template(prompt_template)
    | llm
    | StrOutputParser()
)

In [127]:
summaries = chain.batch(docs, {"max_concurrency": 1})

In [128]:
summary_docs = []
id_key = "doc_id"
index_type = 'summary'
for i, s in enumerate(summaries):
    _id = doc_ids[i]
    doc = docs[i]
    if doc.metadata["is_table"] == 1:
        _doc = Document(page_content=doc.page_content,
                        metadata={"type": index_type, id_key: _id})
        summary_docs.extend([_doc])
        continue
        
    _s = Document(page_content=s, 
                 metadata={"type": index_type, id_key: _id})
    
    summary_docs.extend([_s])

In [134]:
summary_chunk_db = Chroma.from_documents(summary_docs,
                                     embedding_model.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db_summary",
                                     )

In [135]:
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever (empty to start)
summary_retriever = MultiVectorRetriever(
    vectorstore=summary_chunk_db,
    byte_store=store,
    id_key=id_key,
)
summary_retriever.docstore.mset(list(zip(doc_ids, docs)))

In [136]:
query = "出差交通费怎么算？"
summary_retriever.invoke(query)

[Document(metadata={'type': 'ori', 'is_table': 0}, page_content='四、市内交通费\n市内交通费是指工作人员因公出差期间发生的市内交通费用。出差人员的市内交通费按出差的自然（日历）天数计算，每人每天80 元包干使用。由接待单位或其他单位提供交通工具的，不再发放市内交通费。'),
 Document(metadata={'type': 'ori', 'is_table': 1}, page_content='差旅费用标准\n差旅费开支范围包括工作人员临时到常驻地以外地区公务出差所发生的城市间交通费、住宿费、伙食补助费和市内交通费。一、城市间交通费城市间交通费是指工作人员因公到常驻地以外地区出差乘坐火车、轮船、飞机等交通工具所发生的费用。1.出差人员在不影响公务、确保安全的前提下，选乘经济便捷的交通工具。2.出差人员要按照规定等级乘坐交通工具，未按规定乘坐的，超支部分自理。乘坐交通工具的等级见下表：\n<table>\n<tr><td  >级别</td><td  >火车 （含高铁、动车、全列软席列车）</td><td  >轮船 （不包括 旅游船）</td><td  >飞机</td><td  >其他交通工具 （不包括出租 小汽车）</td></tr>\n<tr><td  >享受副部级 待遇及以上 人员</td><td  >火车软席（软座、软卧），高铁/动车商 务座，全列软席列车一等软座</td><td  >一等舱</td><td  >头等舱</td><td  >凭据报销</td></tr>\n<tr><td  >秘书长及副 秘书长</td><td  >火车软席（软座、软卧），高铁/动车一 等座，全列软席列车一等软座</td><td  >二等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n<tr><td  >其余人员</td><td  >火车硬席（硬座、硬卧），高铁/动车二 等座、全列软席列车二等软座</td><td  >三等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n</table>\n备注：\n享受副部级待遇及以上人员出差，因工作需要，随行一人可乘坐同等级交通工具；乘坐飞机的，民航发展基金、燃油附加费可以凭据报销；乘坐飞机、火车、轮船

In [137]:
summary_chunk_db.similarity_search(query, k=2)

[Document(metadata={'doc_id': 'a19a61c6-f004-4cae-9ac1-3de3f861548e', 'type': 'summary'}, page_content='摘要：市内交通费为出差期间每日80元定额，按自然天数计算，若使用接待单位车辆则不发放。\n\n关键词：市内交通费、出差、每人每天80元、自然（日历）天数、接待单位交通工具。'),
 Document(metadata={'doc_id': 'afa8e13d-0d63-452e-a443-214c20ad6fd9', 'type': 'summary'}, page_content='差旅费用标准\n差旅费开支范围包括工作人员临时到常驻地以外地区公务出差所发生的城市间交通费、住宿费、伙食补助费和市内交通费。一、城市间交通费城市间交通费是指工作人员因公到常驻地以外地区出差乘坐火车、轮船、飞机等交通工具所发生的费用。1.出差人员在不影响公务、确保安全的前提下，选乘经济便捷的交通工具。2.出差人员要按照规定等级乘坐交通工具，未按规定乘坐的，超支部分自理。乘坐交通工具的等级见下表：\n<table>\n<tr><td  >级别</td><td  >火车 （含高铁、动车、全列软席列车）</td><td  >轮船 （不包括 旅游船）</td><td  >飞机</td><td  >其他交通工具 （不包括出租 小汽车）</td></tr>\n<tr><td  >享受副部级 待遇及以上 人员</td><td  >火车软席（软座、软卧），高铁/动车商 务座，全列软席列车一等软座</td><td  >一等舱</td><td  >头等舱</td><td  >凭据报销</td></tr>\n<tr><td  >秘书长及副 秘书长</td><td  >火车软席（软座、软卧），高铁/动车一 等座，全列软席列车一等软座</td><td  >二等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n<tr><td  >其余人员</td><td  >火车硬席（硬座、硬卧），高铁/动车二 等座、全列软席列车二等软座</td><td  >三等舱</td><td  >经济舱</td><td  >凭据报销</td></tr>\n</table>\n备注：\n享受副部级待遇及以

## 多索引检索-假设问题检索

In [140]:
from typing import List
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

In [141]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template(
        "你是企业员工助手，熟悉公司考勤和报销标准等规章制度。你的任务是提出在下面文档的内容中可以找到答案的3个假设性问题。:\n\n{doc}, 要求输出为中文，不包含解释性内容，格式列表格式如['问题1'， '问题2', '问题3'] "
    )
    | llm
)

In [143]:
docs[4]

Document(metadata={'type': 'ori', 'is_table': 0}, page_content='。工作时间：星期一至星期四7:55-16:55 星期五7:55-16:152、所有教职工实行考勤打卡制度，工作日内，每天需打卡两次，上午上班一次和下午下班一次。3、教职员工因故(特殊情况除外)晚到或早退，应事先履行请假手续，经批准后方可离校')

In [142]:
chain.invoke(docs[4])

"['教职工在星期一至星期四的工作时间是多久？', '教职员工每天需要打卡几次？', '如果教职员工因故晚到或早退，需要履行什么手续？']"

In [144]:
class HypotheticalQuestions(BaseModel):
    """Generate hypothetical questions."""

    questions: List[str] = Field(..., description="List of questions")

In [145]:
question_docs = []
id_key = "doc_id"
index_type = 'hq'
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    for _ in range(3):
        try:
            hq = chain.invoke(doc)
            res = eval(hq)
            q = HypotheticalQuestions(questions=res)
            
            for i, question in enumerate(q.questions):
                question_docs.extend(
                    [Document(page_content=question, metadata={"type": index_type, id_key: _id})])
            break
        except:
            continue

In [146]:
hq_chunk_db = Chroma.from_documents(question_docs,
                                     embedding_model.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db_hq",
                                     )

In [148]:
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever (empty to start)
hq_retriever = MultiVectorRetriever(
    vectorstore=hq_chunk_db,
    byte_store=store,
    id_key=id_key,
)
hq_retriever.docstore.mset(list(zip(doc_ids, docs)))

In [149]:
query = "出差交通费怎么算？"
hq_retriever.invoke(query)

[Document(metadata={'type': 'ori', 'is_table': 0}, page_content='四、市内交通费\n市内交通费是指工作人员因公出差期间发生的市内交通费用。出差人员的市内交通费按出差的自然（日历）天数计算，每人每天80 元包干使用。由接待单位或其他单位提供交通工具的，不再发放市内交通费。'),
 Document(metadata={'type': 'ori', 'is_table': 0}, page_content='。（加班需提前申请），加班需有打卡记录，无打卡记录支撑的加班时间，不计加班。加班费按照实际加班时长的2倍计算。3、加班以调休等额返还（代替），凡调休人员应填写《请假申请表》，选择“调休”一栏，经所在部门分管领导签字后，交由人事处核实备案')]

In [150]:
query = "出差交通费怎么算？"
hq_chunk_db.similarity_search(query, k=2)

[Document(metadata={'doc_id': 'a19a61c6-f004-4cae-9ac1-3de3f861548e', 'type': 'hq'}, page_content='市内交通费是按照什么方式计算的？'),
 Document(metadata={'doc_id': 'a19a61c6-f004-4cae-9ac1-3de3f861548e', 'type': 'hq'}, page_content='员工因公出差期间，每天的市内交通费用标准是多少？')]

## 融合检索

- pip install rank_bm25
- pip install jieba

In [151]:
import jieba
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [152]:
def jieba_preprocessing_func(text: str):
    return list(jieba.cut(text))

In [153]:
text = "我要请病假100天"
jieba_preprocessing_func(text)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.851 seconds.
Prefix dict has been built successfully.


['我要', '请', '病假', '100', '天']

In [155]:
bm25_retriever = BM25Retriever.from_documents(docs, preprocess_func=jieba_preprocessing_func)
bm25_retriever.k = 3

In [156]:
query = "我要请病假100天"
ret_docs = bm25_retriever.invoke(query)

In [157]:
for doc in ret_docs:
    print("#"*100)
    print(doc.page_content)

####################################################################################################
<table><caption>伙食补助费参考以下标准：</caption>
<tr><td  >地区</td><td  >伙食补助费标准</td></tr>
<tr><td  >西藏、青海、新疆</td><td  >120 元/人、天</td></tr>
<tr><td  >其他省份</td><td  >100 元/人、天</td></tr>
</table>
####################################################################################################
2
<table><caption>伙食补助费参考以下标准：</caption>
<tr><td  >地区</td><td  >伙食补助费标准</td></tr>
<tr><td  >西藏、青海、新疆</td><td  >120 元/人、天</td></tr>
<tr><td  >其他省份</td><td  >100 元/人、天</td></tr>
</table>
####################################################################################################
第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断

In [158]:
query = "婚假如何请"
ret_docs = bm25_retriever.invoke(query)

In [159]:
for doc in ret_docs:
    print("#"*100)
    print(doc.page_content)

####################################################################################################
（1）婚假期间员工工资照发。
（2）为了保证教学活动的顺利进行，提倡婚假安排在寒暑假。
4、丧假：员工因配偶或直系亲属丧亡时，可申请丧假：配偶、父母（公婆、岳父母）、子女及其配偶、祖父母、外祖父母、孙子女（外孙子女）及其配偶、曾祖父母、曾外祖父母丧亡，丧假3 天。
5、产假：符合计划生育政策规定的女性员工可享受产假
####################################################################################################
第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><

In [160]:
embedding_retriever = zhidu_db.as_retriever(search_kwargs={"k": 3})

In [161]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, embedding_retriever], weights=[0.3, 0.7]
)

In [162]:
query = "公司丧假有什么规定？"
related_docs = ensemble_retriever.invoke(query)[:3]

In [163]:
for doc in related_docs:
    print("#"*100)
    print(doc.page_content)

####################################################################################################
（1）婚假期间员工工资照发。
（2）为了保证教学活动的顺利进行，提倡婚假安排在寒暑假。
4、丧假：员工因配偶或直系亲属丧亡时，可申请丧假：配偶、父母（公婆、岳父母）、子女及其配偶、祖父母、外祖父母、孙子女（外孙子女）及其配偶、曾祖父母、曾外祖父母丧亡，丧假3 天。
5、产假：符合计划生育政策规定的女性员工可享受产假
####################################################################################################
第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><

In [170]:
run_rag_pipline(query, related_docs, k=3, context_query_type="doc")



####################################################################################################
query: 公司丧假有什么规定？
context: 上下文1: （1）婚假期间员工工资照发。
（2）为了保证教学活动的顺利进行，提倡婚假安排在寒暑假。
4、丧假：员工因配偶或直系亲属丧亡时，可申请丧假：配偶、父母（公婆、岳父母）、子女及其配偶、祖父母、外祖父母、孙子女（外孙子女）及其配偶、曾祖父母、曾外祖父母丧亡，丧假3 天。
5、产假：符合计划生育政策规定的女性员工可享受产假 

上下文2: 第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>


''

## rerank

In [171]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [172]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [173]:
model_path = './data/llm_app/embedding_models/bge-reranker-base/'

In [174]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
rerank_model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [175]:
rerank_model = rerank_model.cuda()

In [176]:
query = "我要请病假100天"
ret_docs = bm25_retriever.invoke(query)

In [185]:
ret_docs

[Document(metadata={'type': 'ori', 'is_table': 1}, page_content='<table><caption>伙食补助费参考以下标准：</caption>\n<tr><td  >地区</td><td  >伙食补助费标准</td></tr>\n<tr><td  >西藏、青海、新疆</td><td  >120 元/人、天</td></tr>\n<tr><td  >其他省份</td><td  >100 元/人、天</td></tr>\n</table>'),
 Document(metadata={'type': 'ori', 'is_table': 1}, page_content='2\n<table><caption>伙食补助费参考以下标准：</caption>\n<tr><td  >地区</td><td  >伙食补助费标准</td></tr>\n<tr><td  >西藏、青海、新疆</td><td  >120 元/人、天</td></tr>\n<tr><td  >其他省份</td><td  >100 元/人、天</td></tr>\n</table>'),
 Document(metadata={'type': 'ori', 'is_table': 1}, page_content='第三节休假规定\n休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。\n2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(

In [186]:
pairs = []
for doc in ret_docs:
    pairs.append([query, doc.page_content])

In [187]:
import torch
inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)

with torch.no_grad():
    inputs = {key: inputs[key].cuda() for key in inputs.keys()}
    scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

In [188]:
scores

tensor([-4.0480, -4.0110,  0.5134], device='cuda:0')

In [189]:
doc_sort_ids = list((scores.cpu().numpy()*-1).argsort())

In [190]:
doc_sort_ids

[2, 1, 0]

In [193]:
adocs = [np.array(ret_docs)[doc_sort_ids][0]]

In [194]:
run_rag_pipline(query, adocs, k=3, context_query_type="doc")



####################################################################################################
query: 我要请病假100天
context: 上下文1: 第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>
<tr><td></td><td  >八年以上</td><td  >100% </td></tr>
<tr><td></td><td  >不满一年</td><td  >40% </td></tr>
<tr><td  >6 个月以上

''

## 迭代检索增强生成

In [195]:
iter_retgen_prompt_template = """
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要参考提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，可尝试回答 \
问题：{question} 
"{context}"
回答：
"""

In [197]:
def iter_retgen(query, iter_num=2):
    iter_answer = None
    for i in range(iter_num):
        context_query = f"{query}, {iter_answer}" if iter_answer else query
        print("="*50, "context query-", i, "="*50)
        print(context_query)
        
        if i < iter_num - 1:
            iter_answer = run_rag_pipline(query, context_query, k=3, context_query_type="query", 
                                          stream=False, 
                                          prompt_template=iter_retgen_prompt_template,
                                          temperature=1.1)
            print("="*50, "iter-", i, "="*50)
            print(iter_answer)
        else:
            iter_answer = run_rag_pipline(query, context_query, k=3, context_query_type="query", 
                              stream=True, 
                              prompt_template=prompt_template,
                              temperature=0.1)

In [200]:
query = "公司的假有哪些，规定是什么？"
iter_retgen(query)

================================================== context query- 0 ==================================================
公司的假有哪些，规定是什么？


####################################################################################################
query: 公司的假有哪些，规定是什么？
context: 上下文1: 第三节休假规定
休假分以下八种：事假、病假、婚假、丧假、产假、哺乳假、工伤假、调休。1、事假：（1）请假必须严格按照学校规定的请假程序，应由本人以书面形式，应在前两日办理手续，特殊情况应通过电话或者口头请假，应在事假结束于2 个工作日内完成补办请假手续。未经请假或请假未准而擅自离岗者，以旷工论处。（2）事假最小计算单位为半天，事假一次不得超过3 天。（3）事假：基本工资和岗位津贴均按请假天数占实际上班天数比例来算。（4）请假理由不充分或致工作妨碍时，可酌情缩短假期、或延期给假、或不予给假。（5）请假者必须将所任课务或经办事务交待给代理人员，并于请假单内注明。
2、病假：因身体健康问题不能正常工作的员工可申请病假，休假后须提供三级医院开具的病假条或诊断证明。（1）教职工休病假需提前申请。如因情况紧急或突发情况无法请假的，应通过电话或者口头请假，应在病假结束于2 个工作日内补办相关手续。未经请假或请假未准而擅自离岗者，以旷工论处。(2)病假按照工龄系数，对病假日工资进行扣除。
<table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</

## self-RAG
- 使用langgraph来实现；
- LangGraph 通过节点（Nodes）和边（Edges）来定义工作流，其中每个节点可以执行特定的任务，而边则定义了数据如何在这些任务之间流动

![](./data/selfrag.drawio.png)

In [201]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [202]:
from typing import Dict, TypedDict
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph


In [203]:
def rag_retrieve(question, k=3):
    related_docs = zhidu_db.similarity_search(question, k=3)
    context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(related_docs)])
    return related_docs, context

In [205]:
def retrieve(state):

    print("---retrieve---")
    # node input 
    state_dict = state["keys"]
    question = state_dict["question"]
    context_query = state_dict.get("context_query", None)
    query2doc_count = state_dict.get("query2doc_count", 0)
    rewrite_count = state_dict.get("rewrite_count", 0)
    
    # task
    if context_query is not None:
        documents, context = rag_retrieve(context_query, k=3)
    else:
        documents, context = rag_retrieve(question, k=3)
        
    # node output
    return {"keys": {"context": context, 
                     "question": question, 
                     "documents": documents,
                     "query2doc_count": query2doc_count,
                     "rewrite_count": rewrite_count}}

In [206]:
def transform_query2doc(state):
    print("---transform_query2doc---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    context = state_dict["context"]
    query2doc_count = state_dict.get("query2doc_count", 0)
    rewrite_count = state_dict.get("rewrite_count", 0)
    

    context_query = query2doc(question)
    return {"keys": {"context": context,
                     "documents": documents, 
                     "question": question, 
                     "context_query": context_query, 
                     "query2doc_count": query2doc_count+1,
                     "rewrite_count": rewrite_count}}

In [207]:
def transform_query_rewrite(state):
    print("---transform_query---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    context = state_dict["context"]
    query2doc_count = state_dict.get("query2doc_count", 0)
    rewrite_count = state_dict.get("rewrite_count", 0)
    

    context_query = question_rewrite(question)
    
    return {"keys": {"context": context,
                     "documents": documents, 
                     "question": question, 
                     "context_query": context_query,
                     "query2doc_count": query2doc_count,
                     "rewrite_count": rewrite_count+1}}

In [208]:
def generate(state):

    print("---Generate answer---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    query2doc_count = state_dict.get("query2doc_count", 0)
    rewrite_count = state_dict.get("rewrite_count", 0)
    
    # context
    context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(documents)])
    
    # Prompt
    prompt = PromptTemplate(
                        input_variables=["question", "context"],
                        template=prompt_template,)
    llm_prompt = prompt.format(question=query, context=context)

    # LLM
    llm = langchain_llm

    # Chain
    rag_chain = prompt | llm | StrOutputParser()
    
    # Run
    generation = rag_chain.invoke({"context": context, "question": question})
    return {
        "keys": {"context": context, 
                 "question": question, 
                 "documents": documents, 
                 "generation": generation,
                 "query2doc_count": query2doc_count,
                 "rewrite_count": rewrite_count
                }
    }

In [209]:
context_grade_prompt = PromptTemplate(
    template="""您是一名评分员，针对公司规章的问题，负责评估检索到的文档与用户问题的相关性。\n 
    以下是检索到的文档: \n\n {context} \n\n
    以下是用户问题: {question} \n
    如果文档包含与用户问题相关的关键词或语义信息，请将其评为相关. \n
    请给出一个二元分类 'Yes' 或 'No'，以指示该文档是否与问题相关.""",
    input_variables=["context", "question"],
)

context_grade_chain = context_grade_prompt | langchain_llm | StrOutputParser()

In [210]:
answer_useful_prompt = PromptTemplate(
                        template="""您是一名评分员，针对公司规章的问题，负责评估一个答案是否对解决用户问题有用。\n 
                        以下是答案: \n\n {generation} \n\n
                        以下是用户问题: {question} \n
                        请给出一个二元分类 'Yes' 或 'No'，以指示答案是否对解决问题有用.""",
                        input_variables=["generation", "question"],
                    )
answer_useful_chain = answer_useful_prompt | langchain_llm | StrOutputParser()

In [211]:
answer_supported_prompt = PromptTemplate(
                        template="""您是一名评分员，针对公司规章的问题，负责评估一个答案是否基于或者由一组事实支持。\n 
                        以下是事实: \n\n {context} \n\n
                        以下是答案: {generation} \n
                        请给出一个二元分类 'Yes' 或 'No'，以指示答案是否基于或者由一组事实支持.\n
                        不需要前言或解释""",
                        input_variables=["context", "generation"],
                    )
answer_supported_chain = answer_supported_prompt | langchain_llm | StrOutputParser()

In [212]:
def grade_documents(state):

    print("---Determines whether the retrieved documents are relevant to the question---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    context = state_dict["context"]
    query2doc_count = state_dict.get("query2doc_count", 0)
    rewrite_count = state_dict.get("rewrite_count", 0)
    
    # Score
    filtered_docs = []
    retrieve_enhance = "No"
    for d in documents:
        grade = context_grade_chain.invoke({"question": question, "context": d.page_content})
        if  "yes" in grade.lower():
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            retrieve_enhance = "Yes"
            continue
    if query2doc_count > 0:
        retrieve_enhance = "No"
    context = ""
    if len(filtered_docs):
        context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                         for i, doc in enumerate(filtered_docs)])
    return {
        "keys": {
            "context": context,
            "documents": filtered_docs,
            "question": question,
            "run_retrieve_enhance": retrieve_enhance,
            "query2doc_count": query2doc_count,
            "rewrite_count": rewrite_count
        }
    }

In [213]:
def grade_generation_v_documents_and_question(state):
    print("---Determines whether the answer is relevant to the question---")
    state_dict = state["keys"]
    question = state_dict["question"]
    documents = state_dict["documents"]
    context = state_dict["context"]
    query2doc_count = state_dict.get("query2doc_count", 0)
    generation = state_dict["generation"]
    rewrite_count = state_dict.get("rewrite_count", 0)
    
    grade = answer_supported_chain.invoke({"generation": generation, "context": context})
    
    if "yes" in grade.lower():
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_useful_chain.invoke({"question": question, "generation": generation})
        if "yes" in score.lower():
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            if rewrite_count < 1:
                print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
                return "not useful"
            else:
                return "end"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [215]:
def decide_to_generate(state):

    print("---DECIDE TO GENERATE---")
    state_dict = state["keys"]
    question = state_dict["question"]
    filtered_documents = state_dict["documents"]
    retrieve_enhance = state_dict["run_retrieve_enhance"]
    query2doc_count = state_dict.get("query2doc_count", 0)

    if retrieve_enhance == "Yes":
        if query2doc_count > 1:
            return "generate"
        else:
            print("---DECISION:  transform_query2doc---")
            return "transform_query2doc"
    else:
        print("---DECISION: GENERATE---")
        return "generate"

In [216]:
class GraphState(TypedDict):
    keys: Dict[str, any]
workflow = StateGraph(GraphState)

In [217]:
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query2doc", transform_query2doc) # query2doc
workflow.add_node("transform_query_rewrite", transform_query_rewrite) # query2doc

In [218]:
workflow.set_entry_point("retrieve")

In [219]:
workflow.add_edge("retrieve", "grade_documents")

In [220]:
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query2doc": "transform_query2doc",
        "generate": "generate",
    },
)

In [221]:
workflow.add_edge("transform_query2doc", "retrieve")

In [222]:
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": END,
        "useful": END,
        "end": END,
        "not useful": "transform_query_rewrite",
    },
)

In [223]:
workflow.add_edge("transform_query_rewrite", "retrieve")

In [224]:
app = workflow.compile()

In [225]:
query = "那个，我们公司有什么规定来着？"
inputs = {"keys": {"question": query}}

In [226]:
for output in app.stream(inputs):
    for key, value in output.items():
        pass

---retrieve---
---Determines whether the retrieved documents are relevant to the question---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION:  transform_query2doc---
---transform_query2doc---
#################### query2doc
那个，我们公司有什么规定来着？, 您公司的具体规定会根据所在国家或地区的法律法规以及公司自身的管理需求而定。通常包括但不限于：

1. **工作时间与休息**：明确标准工时、加班政策及休假安排。
2. **薪酬福利**：工资发放周期、奖金制度、保险和退休金计划等。
3. **行为准则**：职业道德规范，如禁止歧视、骚扰，要求诚实守信等。
4. **保密协议**：保护公司商业秘密和个人隐私的规定。
5. **健康与安全**：工作场所的安全规定及紧急情况应对措施。
6. **绩效评估**：员工表现评价标准和流程。
7. **培训与发展**：提供职业发展机会和支持的政策。

具体细节请参考您公司的员工手册或向人力资源部门咨询。每家公司的情况不同，上述仅为一般性概述。
#################### query2doc
---retrieve---
---Determines whether the retrieved documents are relevant to the question---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: GENERATE---
---Generate answer---
---Determines whether the answer is relevant to the questio

In [227]:
print(output['generate']['keys']['generation'])

公司规定包括加班需提前申请并有打卡记录，无打卡记录的加班不计时长。加班费按实际加班时间的2倍计算，或以调休形式返还。请假遇紧急情况可口头申请，但需在上班后两天内补办手续，否则按旷工处理。迟到、早退和未经批准离岗视为旷工，并按照相关办法进行处理。


In [228]:
query = "病假要提交什么材料?"
inputs = {"keys": {"question": query}}
for output in app.stream(inputs):
    for key, value in output.items():
        pass

---retrieve---
---Determines whether the retrieved documents are relevant to the question---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION:  transform_query2doc---
---transform_query2doc---
#################### query2doc
病假要提交什么材料?, 病假申请通常需要提交以下材料：

1. 医疗机构出具的诊断证明或病假条，上面应有医生签名和医院公章。
2. 就诊记录、处方单或其他医疗文件（如果有的话）。

请根据公司具体要求准备相关材料，并及时通知直接上级及人力资源部门。不同公司可能有不同的规定，请参照本公司员工手册或向HR咨询详细要求。
#################### query2doc
---retrieve---
---Determines whether the retrieved documents are relevant to the question---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---DECIDE TO GENERATE---
---DECISION: GENERATE---
---Generate answer---
---Determines whether the answer is relevant to the question---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---


In [229]:
print(output['generate']['keys']['generation'])

根据我们公司的规定，申请病假时，您需要提交以下材料：

1. 病历或诊断证明书：由医院出具的正式文件，上面应有医生签名和医院盖章。
2. 就诊记录：包括挂号单、检查报告等就诊过程中的相关资料。
3. 请假条：请在公司内部系统中填写并提交电子版请假申请。

请注意，具体要求可能因病假天数的不同而有所差异。如果您的病假超过三天，请确保提供完整的医疗证明材料。如有任何疑问，欢迎随时与人力资源部门联系。
